# Describing measurements

In [ ]:
import numpy as np
import pandas as pd

import weldx
import weldx.measurement as msm
import weldx.transformations as tf
from weldx import Q_
from weldx.welding.util import sine

## Overview
In this short example we use welding voltage and current measurements to show how to describe and store measurements with associated measurement chains. This includes describing the measurement equipment and its metadata, describing all the relevant transformation steps from raw-data to the final output and the data itself. The final result is a `MeasurementChain` that should be easy to follow and represent the complete data processing pipeline.

## Generating the measurement data
We start by creating some "dummy" datasets that represent the current and voltage measurements.
In a real application, these would be the datasets that we would copy from our measurement equipment (e.g. downloaded form a HKS-WeldQAS, oscilloscope or similar systems).
The values in this dataset represent the actual physical current and voltage data in A and V.

In [ ]:
time = pd.timedelta_range(start="0s", end="10s", freq="1ms")

In [ ]:
I_ts = sine(f=Q_(10, "1/s"), amp=Q_(20, "A"), bias=Q_(300, "A"))
I = I_ts.interp_time(time)

current_data = weldx.TimeSeries(data=I.data, time=time)
current_data

In [ ]:
U_ts = sine(f=Q_(10, "1/s"), amp=Q_(3, "V"), bias=Q_(40, "V"), phase=Q_(0.1, "rad"))
U = U_ts.interp_time(time)

voltage_data = weldx.TimeSeries(data=U.data, time=time)
voltage_data

It is important to note the type and structure of the `current_data` and `voltage_data` datasets:

- they are created as `TimeSeries`
- the data itself is a `pint.Quantity` i.e. a numpy array with associated unit. For the current measurement this is `ampere`, the voltage is given in `volt`. Using quantities is an important core concept of measurements !
- each `TimeSeries` has a `time` dimension and coordinate using numpy datetime formats.

## Equipment and Software
Next, let's define some equipment and software that is used throughout the measurement chain.
We will use and add more information to these objects later.
In out example, two types of hardware equipment are used:

- The [`HKS P1000-S3`](https://hks-prozesstechnik.de/en/sensors-2/) is a standard welding process sensor that detects the welding voltage and current using a hall sensor. The result is output as two analog signals scaled to +/- 10 V.
- The [`Beckhoff ELM3002-0000`](https://www.beckhoff.com/ELM3002/) is a fieldbus AD-converter terminal that picks up the analog signals of the HKS Sensor and transmits them digitally to the control software.

The final piece involved in the measurement chain is the Software used to record, scale and save both the welding current and voltage measurements. We define the software version and name used during the example using built-in ASDF types.

In [ ]:
HKS_sensor = msm.MeasurementEquipment(name="HKS P1000-S3")

BH_ELM = msm.MeasurementEquipment(name="Beckhoff ELM3002-0000")

In [ ]:
from asdf.tags.core import Software

twincat_scope = Software(name="Beckhoff TwinCAT ScopeView", version="3.4.3143")

## Defining a measurement chain: current measurement
Now we define the missing elements of our measurement chain and bundle everything together. A core concept of the chain are `signals` that go in and out of `transformations` which define mathematical operations of the signals, forming the chain.

Each measurement chain starts with a source signal. This is the point where our physical process or quantity is initially detected by a sensor. Since every `MeasurementChain` needs at least a source, we need to provide the corresponding information when creating an instance of this class. 
As shown in the [tutorial about the measurement chain class](measurement_chain.ipynb), there are several ways to do this.
Because we have already defined our source equipment, we will add the `SignalSource` to the `MeasurementEquipment` and use it to create our `MeasurementChain`. 
The necessary parameters to create the `SignalSource` are its `name`, the `output_signal` and the `source_error` representing the uncertainty attached to the signal source. 

For our current measurement, the source outputs an **analog Signal** of unit **V**. 
According to the spec sheet of the sensor the measurement error in this initial step is **0.1 %** which can be documented using the `Error` property (again using quantities).

In [ ]:
current_source = msm.SignalSource(name="Current sensor",
                                  error=msm.Error(Q_(0.1, "percent")), 
                                  output_signal=msm.Signal(signal_type="analog", unit="V"))

HKS_sensor.sources.append(current_source)

Now we can create the current measurement chain. Since there is no recording of this measurement we do not provide any data. 

In [ ]:
welding_current_chain = msm.MeasurementChain.from_equipment(
    name="welding current measurement chain",
    equipment=HKS_sensor,
)

The next step in the chain is picking up the analog voltage signal from our source with the Beckhoff AD converter terminal which transform the signal into an internal signed integer value. The formula describing this linear transformation with input `x` is
```
a * x + b
32768 / (10 V) * x  + 0
```
We express this signal transformation as an analytical formula. Based on the above formula we also define the static parameters `a` and `b` in the `MathematicalExpression`. Note that we use quantities here as well !
Since our result is a dimensionless integer `a` has the unit **1/V** and `b` is dimensionless which we indicate with `""`.

In [ ]:
from weldx.tags.core.mathematical_expression import MathematicalExpression

current_AD_func = MathematicalExpression(
    expression="a * x + b", 
    parameters= dict(a=Q_(32768.0 / 10.0, "1/V"),
                     b=Q_(0.0, ""))
)

Now that we have the transform function we can define our `SignalTransformation`.
Our new Transformation outputs a new **dimensionless** signal of type **digital**.
The Beckhoff AD converter lists the measurement Error at **0.01 %**.

In [ ]:
current_AD_transform = msm.SignalTransformation(
    name="AD current",
    func=current_AD_func,
    type_transformation="AD",
    error=msm.Error(Q_(0.01, "percent")),
)

We add the transformation to our `MeasurementEquipment` representing the Beckhoff AD converter.

In [ ]:
BH_ELM.transformations.append(current_AD_transform)

Next we use the equipment to add the transformation to the measurement chain.

In [ ]:
welding_current_chain.add_transformation_from_equipment(BH_ELM)

Similar to the AD conversion, we add the final step of our signal processing chain: digitally converting the signal to the final physical representation of the welding current. The current calibration formula from our integer values to the real current values is as follows:
```
1000 A / 32768 * x  + 0 A
```
Put into a new `sympy` expression:

In [ ]:
# define current output calibration expression and transformation
current_calib_func = MathematicalExpression(
    "a * x + b",
    parameters=dict(a= Q_(1000.0 / 32768.0, "A"),
                    b= Q_(0.0, "A"))
)

We create the final transformation step:

- the output signal is our final current measurement representation
- we add the our measurement data to this signal !
- we add the software as a meta field to the signal transformation, since the `Software` class currently does not provide the option to store transformations and therefore can't be linked automatically by the `MeasurementChain`

In [ ]:
current_calib_transform = msm.SignalTransformation(
     name="calibration current",
     error=msm.Error(0.0),
     func=current_calib_func,
     meta=twincat_scope,
)

welding_current_chain.add_transformation(
    transformation=current_calib_transform,
    data=current_data
)

Now that our `MeasurementChain` is complete, we can visualize it using its `plot` function.

In [ ]:
welding_current_chain.plot()

Finally the `Measurement` is our measurement chain with another link to the data.

In [ ]:
welding_current = msm.Measurement(
    name="welding current measurement",
    data=[current_data],
    measurement_chain=welding_current_chain,
)

## voltage measurement
We follow the same procedure described in the current measurement here :-)

In [ ]:
voltage_source = msm.SignalSource(name="Voltage sensor",
                                  error=msm.Error(Q_(0.1, "percent")), 
                                  output_signal=msm.Signal(signal_type="analog", unit="V"))

HKS_sensor.sources.append(voltage_source)

welding_voltage_chain = msm.MeasurementChain.from_equipment(
    name="welding voltage measurement",
    equipment=HKS_sensor,
    source_name="Voltage sensor"
) 


voltage_ad_func = MathematicalExpression(
    "a * x + b",
    parameters=dict(a = Q_(32768.0 / 10.0, "1/V"),
                    b = Q_(0.0, ""))
)

voltage_AD_transform = msm.SignalTransformation(
     name="AD voltage",
     error=msm.Error(Q_(0.01, "percent")),
     func=voltage_ad_func,
     type_transformation="AD",
)

BH_ELM.transformations.append(voltage_AD_transform)

welding_voltage_chain.add_transformation_from_equipment(
    equipment=BH_ELM,
    transformation_name="AD voltage"
)


# define voltage output calibration expression and transformation
voltage_calib_func = MathematicalExpression(
    expression="a * x + b",
    parameters=dict(a = Q_(100.0 / 32768.0, "V"),
                    b = Q_(0.0, "V"))
)

voltage_calib_transform = msm.SignalTransformation(
     name="calibration voltage",
     error=msm.Error(0.0),
     func=voltage_calib_func,
     meta=twincat_scope,
)

welding_voltage_chain.add_transformation(
    transformation=voltage_calib_transform,
    data=voltage_data,
)


welding_voltage = msm.Measurement(
    name="welding voltage measurement",
    data=[voltage_data],
    measurement_chain=welding_voltage_chain,
)
welding_voltage_chain.plot()

## Coordinate Systems

Most data does not make much sense without being able to determine where it was recorded in relation to a specimen or other measurement spots. Therefore, we define coordinate systems and their orientations towards each other. The basic principles are already explained in the transformation tutorials, so we will just define some coordinate systems without further explanation. To keep things simple, no time dependent coordinates are considered.

In [ ]:
lcs_specimen_in_root = tf.LocalCoordinateSystem(
    coordinates=Q_(np.asarray([100, 75, 0]), "mm")
)
lcs_flange_in_root = tf.LocalCoordinateSystem(
    orientation=tf.WXRotation.from_euler("x",np.pi / 2).as_matrix(),
    coordinates=Q_(np.asarray([115, -10, 140]), "mm"),
)
lcs_torch_in_flange = tf.LocalCoordinateSystem(
    coordinates=Q_(np.asarray([100, 75, 0]), "mm")
)

coordinate_systems = tf.CoordinateSystemManager("root")
coordinate_systems.add_cs("specimen", "root", lcs_specimen_in_root)
coordinate_systems.add_cs("flange", "root", lcs_flange_in_root)
coordinate_systems.add_cs("torch", "flange", lcs_torch_in_flange)

> **TODO:** Connect data to coordinate systems

## Writing to ASDF
Once we have defined all object we can write them to an ASDF file. To make the file easier to read we place some elements earlier in the tree.

In [ ]:
equipment = [HKS_sensor, BH_ELM]
measurement_data = [current_data, voltage_data]
measurements = [welding_current, welding_voltage]

tree = {
    #"coordinate_systems": coordinate_systems,
    "equipment": equipment,
    "data": measurement_data,
    "measurements": measurements,
}
file = weldx.WeldxFile(tree=tree)

In [ ]:
file.show_asdf_header()